# A Basic Introduction to using APIs for Acquiring Data
---

First, I will use these two libraries, so I should load them:

In [1]:
# Import two libraries
import requests # this handles the getting web data
import pandas as pd # this is pandas, where I've given a short name 'pd'

## Open Notify (who's in Space!)

In [4]:
# Set the variable to the output of the requests.get() function
# Pointed at a very specific domain
resp = requests.get('http://api.open-notify.org/astros.json')
# This is an if command
if resp.status_code==200 : # the condition here is a 200 status code (something like a 404 or 403 would be an error!)
    print("Good GET") # everything that's indented is then run if true
received_dict=resp.json()
# Anything run on the last line will be output to the notebook
received_dict

Good GET


{'people': [{'craft': 'ISS', 'name': 'Mark Vande Hei'},
  {'craft': 'ISS', 'name': 'Pyotr Dubrov'},
  {'craft': 'ISS', 'name': 'Anton Shkaplerov'},
  {'craft': 'Shenzhou 13', 'name': 'Zhai Zhigang'},
  {'craft': 'Shenzhou 13', 'name': 'Wang Yaping'},
  {'craft': 'Shenzhou 13', 'name': 'Ye Guangfu'},
  {'craft': 'ISS', 'name': 'Raja Chari'},
  {'craft': 'ISS', 'name': 'Tom Marshburn'},
  {'craft': 'ISS', 'name': 'Kayla Barron'},
  {'craft': 'ISS', 'name': 'Matthias Maurer'}],
 'message': 'success',
 'number': 10}

* Explaining the output, the braces `{ }` tell us this is a dict data type (a dictionary). 
* Each entry is an `key:value` pair a `'key'` to describe the entry, and a `value` associated with that key. 
   * So the `'message'` key has value `'success'`. 
   * The `'people'` key has a value which is a list `[ ]`
   * Each entry in the `people` list is another dictionary with two keys 
      * A `craft` for the name of the vessel
      * A `name` for the astronaut

Extracting the `people` entry, we just enter the key as an argument for the dictionary. 

In [9]:
spacePeople=received_dict['people']
spacePeople

[{'craft': 'ISS', 'name': 'Mark Vande Hei'},
 {'craft': 'ISS', 'name': 'Pyotr Dubrov'},
 {'craft': 'ISS', 'name': 'Anton Shkaplerov'},
 {'craft': 'Shenzhou 13', 'name': 'Zhai Zhigang'},
 {'craft': 'Shenzhou 13', 'name': 'Wang Yaping'},
 {'craft': 'Shenzhou 13', 'name': 'Ye Guangfu'},
 {'craft': 'ISS', 'name': 'Raja Chari'},
 {'craft': 'ISS', 'name': 'Tom Marshburn'},
 {'craft': 'ISS', 'name': 'Kayla Barron'},
 {'craft': 'ISS', 'name': 'Matthias Maurer'}]

Similarly, for the list we can call any entry by the numerical value (where the first entry is 0)

In [12]:
spacePeople[0]['craft']

'ISS'

In [13]:
spacePeople[6]

{'craft': 'ISS', 'name': 'Raja Chari'}

Often in Python it's good to figure out how long things are, or what data type they are:

In [14]:
print(  len( spacePeople )  )  # len() tells me the length of the listable object
type(spacePeople)  # type() confirms this is a list data type

10


list

The list here is made up of a series of 'dict' (a dictionary) entries, the key-name and value pair for the specific list entry.

Here I'm just going to extract the type for each entry in turn and print it to output:

In [15]:
for entry in spacePeople:
    print(type(entry ))

<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>


Thankfully, it's easy to bring Lists of dictionaries like this into Pandas as a dataframe

In [16]:
df=pd.DataFrame(resp.json()['people'])
df

craft              name
0          ISS    Mark Vande Hei
1          ISS      Pyotr Dubrov
2          ISS  Anton Shkaplerov
3  Shenzhou 13      Zhai Zhigang
4  Shenzhou 13       Wang Yaping
5  Shenzhou 13        Ye Guangfu
6          ISS        Raja Chari
7          ISS     Tom Marshburn
8          ISS      Kayla Barron
9          ISS   Matthias Maurer

Dataframes in python are similar to R, where each column can be a different data type. 

Thankfully, pandas has lots of helper functions built in that can operate on both columns

In [17]:
for spacecraft in df['craft'].unique(): #( for each unique entry in the column)
    print(spacecraft) # print out it's name

ISS
Shenzhou 13


Because you may want to get data via Python, but analyze it in R, it is often convenient to export the dataframe you've constructed to a csv file.

In [ ]:
df.to_csv('CurrentSpacePeople.csv',columns=["craft","name"])

## FBI Most Wanted

Anyone who is making an API available will normally have some documentation about how to make requests. For the FBI most wanted, though fairly minimal, there is documentation [here](https://www.fbi.gov/wanted/api)

I read this, and will use it to access their data.

In [18]:
# Make an empty list
resp_list=[]
# Add the first page of result to the list 
resp_list.append( requests.get('https://api.fbi.gov/wanted/v1/list'))
# get the status code for the page (remember 200 codes are good!)
print(resp_list[0].status_code) # note that in python, lists starts at element zero!

200


In [19]:
type(resp_list[0])

requests.models.Response

The response data is stored in the `.json()` part of the requests response

In [ ]:
resp_list[0].json() # This will be quite long!

Probably easier to just iterate through the data here:

In [21]:
i=1 # initialize variable i to 1
# For each key in the dictionary, print the key name
for key in resp_list[0].json(): 
    print("key "+str(i)+": "+key)
    i=i+1
# Now that this is no longer indented, it's not part of the for loop
print (str(i-1) +" keys printed") # Summarize what we just did

key 1: total
key 2: items
key 3: page
3 keys printed


The page key here is just which page of entries the request got from the website

In [28]:
len(resp_list[0].json()['items'])

20

In [23]:
print("There are "+ str( resp_list[0].json()['total'] )+ " total entries in the MostWanted List")
print("But there are only "+str(  len(resp_list[0].json()['items']) ) + " entries in the Items list!")

There are 946 total entries in the MostWanted List
But there are only 20 entries in the Items list!


The reason for this, is that our get request only populated the first page. From reading the documentation, we can pass a parameter to the API to ask for a different page:

In [29]:
resp_list.append(requests.get('https://api.fbi.gov/wanted/v1/list', params={'page': 2}))
print(resp_list[1].status_code)

200


Let's get multiple pages using a for loop... but before we do, let's look at one intricacy of python:

In [32]:
for ii in range(0,5): # range(0,5) would be similar
    print(ii)

0
1
2
3
4


So the range command here runs over 5 total entries, but because it starts at zero, the last entry is 4.

So we're going to need how many pages?

In [33]:
resp_list[0].json()['total']/ len( resp_list[0].json()['items'] )

47.3

Initializing our list:

In [34]:
resp_list=[]

So we'll explicitly tell it to look through pages 1 to 8

In [35]:
for ii in range(1,25): # for 1 and 15 (note it doesn't include 16) do the following: 
    resp_list.append(requests.get('https://api.fbi.gov/wanted/v1/list', params={'page': ii}))
    if resp_list[ii-1].status_code==200: print("Page "+str(ii)+" captured") # this is indented, so it's within the for loop

Page 1 captured
Page 2 captured
Page 3 captured
Page 4 captured
Page 5 captured
Page 6 captured
Page 7 captured
Page 8 captured
Page 9 captured
Page 10 captured
Page 11 captured
Page 12 captured
Page 13 captured
Page 14 captured
Page 15 captured
Page 16 captured
Page 17 captured
Page 18 captured
Page 19 captured
Page 20 captured
Page 21 captured
Page 22 captured
Page 23 captured
Page 24 captured


In [36]:
resp_list # let's check they all have 200 codes!

[<Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>]

I'm going to append all the entries in the list together as a dataframe:

In [38]:
df_list=[] # create empty list
for resp in resp_list:
    df_list.append( pd.DataFrame(resp.json()['items']) ) # append the items entry in the json response as a new entry in the list
# No indent, so this is after the for loop...
dfJoined=pd.concat(df_list, ignore_index=True) # Join all the separate dataframes in the list together, ingnoring the separate indices...
dfJoined

files  \
0    [{'url': 'https://www.fbi.gov/wanted/vicap/hom...   
1    [{'url': 'https://www.fbi.gov/wanted/seeking-i...   
2    [{'url': 'https://www.fbi.gov/wanted/seeking-i...   
3    [{'url': 'https://www.fbi.gov/wanted/cei/kenne...   
4    [{'url': 'https://www.fbi.gov/wanted/murders/j...   
..                                                 ...   
475  [{'url': 'https://www.fbi.gov/wanted/cyber/iva...   
476  [{'url': 'https://www.fbi.gov/wanted/cyber/ale...   
477  [{'url': 'https://www.fbi.gov/wanted/seeking-i...   
478  [{'url': 'https://www.fbi.gov/wanted/murders/m...   
479  [{'url': 'https://www.fbi.gov/wanted/murders/c...   

                                             age_range  \
0                        19 years old at time of death   
1    Approximately 18 to 40 years old but may appea...   
2                                                 None   
3                                                 None   
4                                                 None   
..                                                 ...   
475                                               None   
476                                               None   
477                                               None   
478                                               None   
479                                               None   

                                      uid             weight  \
0        7241019564ec4aa69ef0b87d3ac25456         115 pounds   
1        e57593ca70734204957ed6906d3051e7               None   
2    51cb961b-fb58-4c6a-a89b-cd34c1df04b7               None   
3        286c7f6c919d4c2885f46087a26e554b         149 pounds   
4        57759f4939ac449fbc6058cc3456fe7c         150 pounds   
..                                    ...                ...   
475      a3f70fd05e6e435cbb7ebe1d9201524e               None   
476      5ab231b51f9841d589ae05c3a19bbd6a               None   
477      459fcd662ee849fe9df91f59de93f83f         200 pounds   
478      367dfdc4df89411aa6277af54277ee2b         165 pounds   
479      5246151b4f49443c8ce3d2b4f6bc9e24  122 to 180 pounds   

                                           occupations   field_offices  \
0                                                 None            None   
1                                                 None  [indianapolis]   
2                                                 None       [newyork]   
3                                                 None       [norfolk]   
4                                                 None   [albuquerque]   
..                                                 ...             ...   
475  [Officer in the Russian Federation’s Main Inte...            None   
476  [Officer in the Russian Federation’s Main Inte...            None   
477                                               None   [albuquerque]   
478                                               None    [losangeles]   
479                                               None    [losangeles]   

    locations                                        reward_text   hair  \
0        None                                               None  brown   
1        None                                               None  brown   
2        None  The FBI is offering a reward of up to $100,000...   None   
3        None  The FBI is offering a reward of up to $1,000 f...  blond   
4        None  The FBI is offering a reward of up to $10,000 ...  black   
..        ...                                                ...    ...   
475      None                                               None  brown   
476      None                                               None  brown   
477      None  The FBI is offering a reward of up to $5,000 f...  black   
478      None  The FBI is offering a reward of up to $5,000 f...  black   
479      None  The FBI is offering a reward of up to $5,000 f...  black   

           ncic  ...                                                url  \
0          None  ... 

In [39]:
list_of_lists=dfJoined["field_offices"].to_list()
field_office_list=set() # a set is an unordered list with no repeat entries...
for sublist in list_of_lists: 
    if sublist is not None: # If the sublist has some entries
        for office in sublist: # for each office in the sublist
            if office is not None: field_office_list.add(office) # if it isn't None type, add to the set
field_office_list # print the set at the end!

{'albany',
 'albuquerque',
 'atlanta',
 'baltimore',
 'boston',
 'buffalo',
 'charlotte',
 'chicago',
 'cincinnati',
 'cleveland',
 'columbia',
 'dallas',
 'denver',
 'detroit',
 'elpaso',
 'honolulu',
 'houston',
 'indianapolis',
 'jackson',
 'jacksonville',
 'kansascity',
 'lasvegas',
 'littlerock',
 'losangeles',
 'louisville',
 'miami',
 'milwaukee',
 'minneapolis',
 'mobile',
 'newark',
 'newyork',
 'norfolk',
 'oklahomacity',
 'omaha',
 'philadelphia',
 'phoenix',
 'pittsburgh',
 'portland',
 'richmond',
 'sacramento',
 'saltlakecity',
 'sanantonio',
 'sandiego',
 'sanfrancisco',
 'sanjuan',
 'seattle',
 'stlouis',
 'tampa',
 'washingtondc'}

We can be more precise here, where we could have asked the API about the pittsburgh field office specifically as a parameter:

In [40]:
response = requests.get('https://api.fbi.gov/wanted/v1/list', params={
    'field_offices': 'pittsburgh'
})
pittsburghList=response.json()['items']
len(pittsburghList)

18

In [41]:
pghFBI=pd.DataFrame(pittsburghList)
pghFBI[ ['title','images','description'] ] 

title  \
0               SUSAN GAIL CARTER   
1           NATASHA "ALEX" CARTER   
2                       WEN XINYU   
3                       WANG DONG   
4                      GU CHUNHUI   
5                    HUANG ZHENYU   
6                    SUN KAILIANG   
7   EVGENIY MIKHAILOVICH BOGACHEV   
8              BARRY JAMES DEEMER   
9    ANATOLIY SERGEYEVICH KOVALEV   
10                   DELBERT PATE   
11        IGOR OLEGOVICH TURASHEV   
12    MAKSIM VIKTOROVICH YAKUBETS   
13                GOZNYM SUBJECTS   
14                BOYUSEC HACKERS   
15               KATHRYN MENENDEZ   
16                SARAH RAE BOEHM   
17                 KAREN S. ADAMS   

                                               images  \
0   [{'caption': None, 'large': 'https://www.fbi.g...   
1   [{'caption': None, 'large': 'https://www.fbi.g...   
2   [{'caption': None, 'large': 'https://www.fbi.g...   
3   [{'caption': None, 'large': 'https://www.fbi.g...   
4   [{'caption': None, 'large': 'https://www.fbi.g...   
5   [{'caption': None, 'large': 'https://www.fbi.g...   
6   [{'caption': None, 'large': 'https://www.fbi.g...   
7   [{'caption': None, 'large': 'https://www.fbi.g...   
8   [{'caption': None, 'large': 'https://www.fbi.g...   
9   [{'caption': None, 'large': 'https://www.fbi.g...   
10  [{'caption': None, 'large': 'https://www.fbi.g...   
11  [{'caption': None, 'large': 'https://www.fbi.g...   
12  [{'caption': None, 'large': 'https://www.fbi.g...   
13  [{'caption': None, 'large': 'https://www.fbi.g...   
14  [{'caption': None, 'large': 'https://www.fbi.g...   
15  [{'caption': 'Kathryn Menendez', 'large': 'htt...   
16  [{'caption': 'Sarah Rae Boehm', 'large': 'http...   
17  [{'caption': None, 'large': 'https://www.fbi.g...   

                                          description  
0            August 8, 2000\r\nBeckley, West Virginia  
1            August 8, 2000\r\nBeckley, West Virginia  
2   Conspiring to Commit Computer Fraud; Accessing...  
3   Conspiring to Commit Computer Fraud; Accessing...  
4   Conspiring to Commit Computer Fraud; Accessing...  
5   Conspiring to Commit Computer Fraud; Accessing...  
6   Conspiring to Commit Computer Fraud; Accessing...  
7   Conspiracy to Participate in Racketeering Acti...  
8   Unlawful Flight to Avoid Prosecution - Crimina...  
9   Conspiracy to Commit an Offense Against the Un...  
10  Conspiracy to Distribute and Possess with Inte...  
11  Conspiracy; Conspiracy to Commit Fraud; Wire F...  
12  Conspiracy; Conspiracy to Commit Fraud; Wire F...  
13  COMPUTER FRAUD CONSPIRACY; WIRE AND BANK FRAUD...  
14  Conspiracy to Commit Computer Fraud and Abuse;...  
15  Murder Victim\r\nAlliance, Portage County, Ohi...  
16  Murder Victim\r\nAlliance, Portage County, Ohi...  
17    March 11, 2007\r\nHancock County, West Virginia

There are 18 entries (so 0 to 18 in the list)

In [42]:
imgList=[]
imgLoc=pghFBI[["title","images"]]
for index, row in imgLoc.iterrows():
    imgList.append( {'name':row['title'],'thumb':row['images'][0]['thumb']})

In [44]:
imgList=[]
imgLoc=pghFBI[["title","images"]]
for index, row in imgLoc.iterrows(): # Here I'm iterating over each row in the dataframe
    imgList.append( {'name':row['title'],'thumb':row['images'][0]['thumb']})
# Here I'll use a library called IPython to display the output in a web format:
from IPython.display import Image, display
# Now use Ipython to display them
for img in imgList[ : 5]: # Here I'm getting the first 3 images
    display(img['name'],Image(url=img['thumb'], width=150))

'SUSAN GAIL CARTER'

'NATASHA "ALEX" CARTER'

'WEN XINYU'

'WANG DONG'

'GU CHUNHUI'

___
## Bing Search
This code requires an API key (which is like a password) to get access. I signed up for an account (and a $100 credit, student signup [here](https://azure.microsoft.com/en-us/free/students/https://azure.microsoft.com/en-us/free/students/)), and set up this access account under the *free* tier. There is documentation for the API [here](https://docs.microsoft.com/en-us/azure/cognitive-services/bing-web-search/quickstarts/python) (in fact I copied some of their code!)

Here I just read in my key from a file (that you don't have!)

In [45]:
f = open("../../.bing.txt", "r") # here I'm just reading in my API key stored two directories down
# Open the file, read the authorization key, strip the whitespace
bingKey=f.read().rstrip()
f.close()

In [46]:
server = "https://api.bing.microsoft.com/v7.0/search"
search_term = "Alistair Wilson MQE"
headers = {"Ocp-Apim-Subscription-Key": bingKey} # Here I'm including my API key to get permission to use this resource
params = {"q": search_term, "textDecorations": True, "textFormat": "HTML" }
resp = requests.get(server,
                    headers=headers, params=params
                   )
search_results = resp.json()

In [47]:
print(resp.status_code) # This should be 200

200


In [48]:
i=1
for key in search_results: 
    print("key "+str(i)+": "+key)
    i=i+1

key 1: _type
key 2: queryContext
key 3: webPages
key 4: images
key 5: rankingResponse


In [49]:
type(search_results["webPages"])

dict

In [50]:
webResults=search_results["webPages"]
i=1
for key in webResults: 
    print("key "+str(i)+": "+key)
    i=i+1

key 1: webSearchUrl
key 2: totalEstimatedMatches
key 3: value
key 4: someResultsRemoved


In [51]:
webResults["totalEstimatedMatches"]

886000

The main results from this query are in the `"value"` key

In [52]:
pd.DataFrame(webResults["value"])

id  \
0  https://api.bing.microsoft.com/api/v7/#WebPages.0   
1  https://api.bing.microsoft.com/api/v7/#WebPages.1   
2  https://api.bing.microsoft.com/api/v7/#WebPages.2   
3  https://api.bing.microsoft.com/api/v7/#WebPages.3   
4  https://api.bing.microsoft.com/api/v7/#WebPages.4   
5  https://api.bing.microsoft.com/api/v7/#WebPages.5   
6  https://api.bing.microsoft.com/api/v7/#WebPages.6   
7  https://api.bing.microsoft.com/api/v7/#WebPages.7   
8  https://api.bing.microsoft.com/api/v7/#WebPages.8   
9  https://api.bing.microsoft.com/api/v7/#WebPages.9   

                                                name  \
0  Alistair Wilson, Ph.D. | Master of Science in ...   
1  Alistair Wilson murder: Cops probing mystery e...   
2  People | Master of Science in Quantitative Eco...   
3  Alastair Wilson murder: Police issue new descr...   
4  The Murder of Alistair Wilson - Preliminary ca...   
5  Alistair Wilson murder: Son recalls Nairn bank...   
6        Alistair Wilson - thespiritwithinpoetry.com   
7  Alistair Wilson | City A.M., Surrey Advertiser...   
8  Alistair Wilson - IT Infrastr.. - The Auditor-...   
9  Fresh appeal in the Alistair Wilson (Highlands...   

                                                 url  \
0                https://www.mqe.pitt.edu/people/ant   
1  https://bluzz.org/alistair-wilson-murder-cops-...   
2                    https://www.mqe.pitt.edu/people   
3  https://guernseypress.com/news/uk-news/2022/03...   
4   http://zakmartin.com/alistair_wilson_murder.html   
5  https://www.bbc.com/news/uk-scotland-highlands...   
6  http://thespiritwithinpoetry.com/alistair_wils...   
7               https://muckrack.com/alistair-wilson   
8  https://www.zoominfo.com/p/Alistair-Wilson/-16...   
9  https://www.reddit.com/r/UnresolvedMysteries/c...   

                                        thumbnailUrl  isFamilyFriendly  \
0  https://www.bing.com/th?id=OIP.1zwdTVG2wYc1i7l...              True   
1                                                NaN              True   
2  https://www.bing.com/th?id=OIP.zHfie4h80jFLO8k...              True   
3                                                NaN              True   
4                                                NaN              True   
5                                                NaN              True   
6                                                NaN              True   
7                                                NaN              True   
8                                                NaN              True   
9                                                NaN              True   

                                          displayUrl  \
0         https://www.<b>mqe</b>.pitt.edu/people/ant   
1  https://bluzz.org/<b>alistair</b>-<b>wilson</b...   
2      https://www.<b>mqe</b>.pitt.edu/<b>people</b>   
3  https://guernseypress.com/news/uk-news/2022/03...   
4   zakmartin.com/<b>alistair_wilson</b>_murder.html   
5  https://<b>www.bbc.com</b>/news/uk-scotland-hi...   
6  thespiritwithinpoetry.com/<b>alistair</b>_<b>w...   
7  https://muckrack.com/<b>alistair</b>-<b>wilson...   
8  https://www.zoominfo.com/p/<b>Alistair</b>-<b>...   
9  https://<b>www.reddit.com</b>/.../comments/1of...   

                                             snippet  \
0  Dietrich School of Arts and Sciences Departmen...   
1  <b>Alistair</b> <b>Wilson</b> was just 30 when...   
2  The<b> University of Pittsburgh</b> is among t...   
3  <b>Alastair</b> <b>Wilson</b> murder: Police i...   
4  <b>Alistair Wilson</b>&#39;s change of job <b>...   
5  <b>Alistair Wilson murder:</b> Son recalls Nai...   
6  <b>Alistair</b> <b>Wilson</b>, 30, was shot in...   
7  By <b>Alistair</b> <b>Wilson</b> | Financial A...   
8  View <b>Alistair</b> <b>Wilson</b>&#39;s busin...   
9  <b>Alistair</b> <b>Wilson</b>, married with tw...   

                dateLastCrawled language  isNavigational  
0  2022-03-12T10:19:00.0000000Z       en           False  
1  2022-03-09T00:08:00.0000000Z 

Microsoft's help documentation also gave me the following snippet which uses IPython to display this

In [53]:
from IPython.display import HTML

rows = "\n".join(["""<tr>
                       <td><a href=\"{0}\">{1}</a></td>
                       <td>{2}</td>
                     </tr>""".format(v["url"], v["name"], v["snippet"])
                  for v in search_results["webPages"]["value"]])
HTML("<table>{0}</table>".format(rows))

___
## Canvas API example

This is another API where you need an access key. Again, I've stored it locally, where tehnically, with this one, you could change anything I have access to in Canvas!

As a student in Canvas I believe you can create your own API key in the settings menu.

In [54]:
f = open("../../.canvas.txt", "r")
# Open the file, read the key, strip the whitespace
canvasKey=f.read().rstrip()
f.close()

See [Canvas API reference](canvas.instructure.com/doc/api) for details on the set up here

In [55]:
server = "https://canvas.instructure.com/api/v1/courses"
headers = {"Authorization": "Bearer " + canvasKey}
params = {"textFormat": "HTML",'per_page': 25}
resp = requests.get(server,
                    headers=headers
                   )
if resp.status_code==200 : print("Good GET") 
print( type(resp.json() ) )
print( type(resp.json()[0])  ) 

Good GET
<class 'list'>
<class 'dict'>


This returns a list of entries, where the first entry in that list is a dictionary. 

Let's look at the keys in that dictionary! But to do this let's define a function to do it for us, so we can use this function repeatedly.

In [56]:
def keyPrintFunction(dictIn) : # Deine a function (everything within the function needs to be indented one level)
    # take as input a dict
    if type(dictIn) is dict: # this if line checks that it is a dict input
        typeList=[] # initialize a list
        for key in dictIn:
            # for each entry in the dictIn variable find its type
            typeList.append(type(dictIn[key]).__name__) 
            # The __name__ is just to make sure this variable is a string
        # Now outside of the loop, make a dataframe out of the typeList, where we set the index to the keys 
        dfX = pd.DataFrame( data={'dataType': typeList}, index=list(  dictIn.keys()   )   )
        return dfX # This tells the function to return the dataframe as the output
    else : # All the above was in the if loop, this else line just outputs a message 
        print("Input not a dict type")
        
# The function is ended whenever we're outside the first level of indentation
# So anything you write down here won't be part of the function


But when we run this, it gets captured by our Else...

In [57]:
keyPrintFunction(resp.json())

Input not a dict type


Because the JSON data is instead:

In [58]:
type(resp.json())

list

So let's apply out function to the first entry in our list:

In [59]:
keyPrintFunction(resp.json()[0])

dataType
id                                         int
root_account_id                            int
account_id                                 int
name                                       str
enrollment_term_id                         int
uuid                                       str
start_at                              NoneType
grading_standard_id                   NoneType
is_public                                 bool
created_at                                 str
course_code                                str
default_view                               str
license                                    str
grade_passback_setting                NoneType
end_at                                NoneType
public_syllabus                           bool
public_syllabus_to_auth                   bool
storage_quota_mb                           int
is_public_to_auth_users                   bool
homeroom_course                           bool
course_color                          NoneType
friendly_name                         NoneType
apply_assignment_group_weights            bool
locale                                     str
calendar                                  dict
time_zone                                  str
blueprint                                 bool
template                                  bool
enrollments                               list
hide_final_grades                         bool
workflow_state                             str
restrict_enrollments_to_course_dates      bool
overridden_course_visibility               str

In [60]:
# Let's convert the entire output into a data frame with pandas
courseList=pd.DataFrame( resp.json() )
# And now I'm asking it to display a subset of the columns in the list [ "name" , "id"]
courseList[   [ "name"  , "id"]    ] 

name                  id
0  2191_ECON_0150_SEC1010_QUANTITATIVE ANALYSIS F...  139970000000013234
1  2191_ECON_2220_SEC1030_TOPICS IN EXPERIMENTAL ...  139970000000015671
2      2194_ECON_0150_SEC1010_ECONOMIC DATA ANALYSIS  139970000000008812
3      2194_ECON_0150_SEC1020_ECONOMIC DATA ANALYSIS  139970000000008855
4        2211 ECON 2821 SEC1010 QUANTITATIVE METHODS  139970000000046646
5  2217 CMPINF 2130 SEC1100 THE ART OF DATA VISUA...  139970000000124179
6                        Alistair Wilson Demo Course  139970000000063599
7                                  Exp. Seminar 2022  139970000000132892
8                                       Experimental  139970000000046717
9                                Experimental (2021)  139970000000109528

So our course is not in that list (it's on the next page)

In [61]:
# get the course id in Canvas for our course
mqe_course_id=139970000000132841

From here I simple switch to a different part of the API, where I query the enrollments in this particular course id

In [62]:
server = "https://canvas.instructure.com/api/v1/courses/"
headersIn = {"Authorization": "Bearer " +canvasKey}
# The request string i've assembled come from looking at the API documentation at Canvas
resp = requests.get(  server+"/"+str(mqe_course_id)+"/enrollments",
                    headers=headersIn
                   )
# Return the output as a panas dataframe
studentList = pd.DataFrame(resp.json())
if resp.status_code==200 : print("Good GET") 

Good GET


In [63]:
print(  type(resp.json())  )
print(  type(resp.json()[0])  )

<class 'list'>
<class 'dict'>


So the `resp.json()[0]` variable is a dictionary, so we can apply the function we wrote above to it to get the variable types:

In [64]:
len(resp.json())

10

In [65]:
keyPrintFunction(resp.json()[0])

dataType
course_id                                int
id                                       int
user_id                                  int
course_section_id                        int
root_account_id                          int
type                                     str
created_at                               str
updated_at                               str
associated_user_id                  NoneType
start_at                            NoneType
end_at                              NoneType
limit_privileges_to_course_section      bool
enrollment_state                         str
role                                     str
role_id                                  int
last_activity_at                         str
last_attended_at                    NoneType
total_activity_time                      int
grades                                  dict
sis_account_id                           str
sis_course_id                            str
course_integration_id               NoneType
sis_section_id                           str
section_integration_id              NoneType
sis_user_id                              str
html_url                                 str
user                                    dict

Looking at the objects, there's actually another dictionary embedded within the `"user"` value of this dictionary, so we load that:

In [66]:
resp.json()[8]

{'course_id': 139970000000132841,
 'id': 139970000001286690,
 'user_id': 139970000000124364,
 'course_section_id': 139970000000130186,
 'root_account_id': 139970000000000001,
 'type': 'StudentEnrollment',
 'created_at': '2021-11-13T10:15:27Z',
 'updated_at': '2022-03-03T01:07:04Z',
 'associated_user_id': None,
 'start_at': None,
 'end_at': None,
 'limit_privileges_to_course_section': False,
 'enrollment_state': 'active',
 'role': 'StudentEnrollment',
 'role_id': 139970000000000003,
 'last_activity_at': '2022-03-14T11:25:22Z',
 'last_attended_at': None,
 'total_activity_time': 5481,
 'grades': {'html_url': 'https://canvas.instructure.com/courses/139970000000132841/grades/139970000000124364',
  'current_grade': None,
  'current_score': None,
  'final_grade': None,
  'final_score': None,
  'unposted_current_score': None,
  'unposted_current_grade': None,
  'unposted_final_score': None,
  'unposted_final_grade': None},
 'sis_account_id': 'ARTSC',
 'sis_course_id': '2224_ECON_2823_SEC1020',

In [67]:
keyPrintFunction(resp.json()[0]["user"])

dataType
id                   int
name                 str
created_at           str
sortable_name        str
short_name           str
sis_user_id          str
integration_id  NoneType
root_account         str
login_id             str

And now I just load that information!

In [68]:
# initialize the list
studentList=[]
# for each record in the enrollments json responds
for record in resp.json():
    # print the short_name field
    print(record["user"]["short_name"])
    # append the short name to the studentList
    studentList.append(record["user"]["short_name"])

Rachael Agnello
Shyh Chean Chea
Semora Council
Carolyn Cullen
Liam Flood
Michael Gobreski
Olumayowa Idowu
Joseph Karbowski
Kiersten Kochanowski
Samuel Kusic


In [69]:
# import the random library
import random 
# use it to print a random student name
print(random.choice(studentList))

Kiersten Kochanowski


In [72]:
print(random.choice(studentList))

Shyh Chean Chea


## Prebuilt Libraries for using APIs
---

Often, if an API is popular, others have made libraries to engage with it and get data. Here I just searched for "Canvas API Python"

In [73]:
# I already installed it, but Jupyter has several `magics` for doing other things
## Here i'm installing the cavas API library to this Python Kernel
%pip install canvasapi

Note: you may need to restart the kernel to use updated packages.


In [74]:
# load in the library
# Import the Canvas class
from canvasapi import Canvas
import random # I need this library to Randomize later

canvasUrl='https://canvas.instructure.com'
# Initialize a new Canvas object
canvas = Canvas(canvasUrl, canvasKey)

In [75]:
type(canvas)

canvasapi.canvas.Canvas

In [76]:
alistair=canvas.get_current_user()
logins=alistair.get_user_logins()
for login in logins:
    print(login)

139970000000020610 (ALISTAIR)


In [77]:
courseList=alistair.get_courses()
pd.DataFrame([{'id': course.id,'created': course.created_at_date, 'name': course.name } for course in courseList if course.enrollments[0]['type']=='teacher'])

id                   created  \
0   139970000000013234 2019-11-06 21:58:01+00:00   
1   139970000000015671 2019-11-06 22:00:55+00:00   
2   139970000000008812 2019-11-06 21:52:09+00:00   
3   139970000000008855 2019-11-06 21:52:15+00:00   
4   139970000000046646 2020-04-23 07:05:25+00:00   
5   139970000000063599 2020-07-17 17:49:26+00:00   
6   139970000000132892 2021-10-01 09:06:14+00:00   
7   139970000000046717 2020-04-23 07:05:28+00:00   
8   139970000000109528 2021-03-02 08:08:30+00:00   
9   139970000000077549 2020-10-15 09:04:49+00:00   
10  139970000000132841 2021-10-01 09:06:13+00:00   
11  139970000000046481 2020-04-23 07:05:17+00:00   

                                                 name  
0   2191_ECON_0150_SEC1010_QUANTITATIVE ANALYSIS F...  
1   2191_ECON_2220_SEC1030_TOPICS IN EXPERIMENTAL ...  
2       2194_ECON_0150_SEC1010_ECONOMIC DATA ANALYSIS  
3       2194_ECON_0150_SEC1020_ECONOMIC DATA ANALYSIS  
4         2211 ECON 2821 SEC1010 QUANTITATIVE METHODS  
5                         Alistair Wilson Demo Course  
6                                   Exp. Seminar 2022  
7                                        Experimental  
8                                 Experimental (2021)  
9                                       MQE Spring 1.  
10                                    MQE Spring 2022  
11                                         UG Metrics

In [78]:
mqeCourse=canvas.get_course(139970000000132841)

In [79]:
studentUsers = mqeCourse.get_users(enrollment_type='student')

Here I just make a list of the students, and grab their picture if one is set in Canvas:

In [80]:
studentListIn=[[student.short_name, student.get_profile()['avatar_url'], student.email] for student in studentUsers]
studentList=[]
for student in studentListIn:
        if student[1]=="https://canvas.instructure.com/images/messages/avatar-50.png":
            studentList.append([student[0],"../../img/Avatar_"+ student[2] +".svg",student[2]])
        else:
            studentList.append(student)

Now I create a function to draw k random students from the list of students in the class, where I'm going to use Python to display this!

In [81]:
from IPython.display import display_markdown
def random_student(k):
    selStudents=random.sample(studentList,k)
    strOut="| # | Student | Img | \n | --- | --- | --- |\n"
    i=1
    for student in selStudents:
        strOut=strOut+"| "+ str(i) + "| **" + student[0] + "** | <img src="+student[1]+ " width='128' height='100'>\n"
        i=i+1
    return display_markdown(strOut , raw=True)

In [83]:
random_student(2)

| # | Student | Img | 
 | --- | --- | --- |
| 1| **Joseph Karbowski** | <img src=https://canvas.pitt.edu/images/thumbnails/6567550/t588gMPvmeIhrVHM9xZGdEIbeYZnB8ekyTRLrRKS width='128' height='100'>
| 2| **Dorcas Munoko** | <img src=https://canvas.pitt.edu/images/thumbnails/6598362/4iCYVw6IbpbhY26RzFi4B2B0eJRNj5m173tguuOx width='128' height='100'>


___
## US Census

This one also requires an access token, but they'll email you one if you just ask for it!

You can do that [US Census website](https://www.census.gov/data/developers/data-sets.html), as well as see the more extensive documentation of their API.

In [84]:
f = open("../../census.txt", "r")
# Open the file, read the authorization key, strip the whitespace
censusKey=f.read().rstrip()
f.close()

In [85]:
server = "https://api.census.gov/data/2014/pep/natstprc"
paramsIn = {"get": "STNAME,POP","for" : "state:*", "DATE_" : 7, "key": censusKey}
resp = requests.get(server,
                    params=paramsIn
                   )
if resp.status_code==200 : print("Good GET") 
print( type(resp.json() ) )
print( type(resp.json()[0])  ) 

Good GET
<class 'list'>
<class 'list'>


In [86]:
headColumn=resp.json()[0] # the first entry here is the column heads
dfState=pd.DataFrame(resp.json()[1:])
dfState.columns=headColumn
dfState

STNAME       POP DATE_ state
0                    Alabama   4849377     7    01
1                     Alaska    736732     7    02
2                    Arizona   6731484     7    04
3                   Arkansas   2966369     7    05
4                 California  38802500     7    06
5                   Colorado   5355866     7    08
6                Connecticut   3596677     7    09
7                   Delaware    935614     7    10
8       District of Columbia    658893     7    11
9                    Florida  19893297     7    12
10                   Georgia  10097343     7    13
11                    Hawaii   1419561     7    15
12                     Idaho   1634464     7    16
13                  Illinois  12880580     7    17
14                   Indiana   6596855     7    18
15                      Iowa   3107126     7    19
16                    Kansas   2904021     7    20
17                  Kentucky   4413457     7    21
18                 Louisiana   4649676     7    22
19                     Maine   1330089     7    23
20                  Maryland   5976407     7    24
21             Massachusetts   6745408     7    25
22                  Michigan   9909877     7    26
23                 Minnesota   5457173     7    27
24               Mississippi   2994079     7    28
25                  Missouri   6063589     7    29
26                   Montana   1023579     7    30
27                  Nebraska   1881503     7    31
28                    Nevada   2839099     7    32
29             New Hampshire   1326813     7    33
30                New Jersey   8938175     7    34
31                New Mexico   2085572     7    35
32                  New York  19746227     7    36
33            North Carolina   9943964     7    37
34              North Dakota    739482     7    38
35                      Ohio  11594163     7    39
36                  Oklahoma   3878051     7    40
37                    Oregon   3970239     7    41
38              Pennsylvania  12787209     7    42
39              Rhode Island   1055173     7    44
40            South Carolina   4832482     7    45
41              South Dakota    853175     7    46
42                 Tennessee   6549352     7    47
43                     Texas  26956958     7    48
44                      Utah   2942902     7    49
45                   Vermont    626562     7    50
46                  Virginia   8326289     7    51
47                Washington   7061530     7    53
48             West Virginia   1850326     7    54
49                 Wisconsin   5757564     7    55
50                   Wyoming    584153     7    56
51  Puerto Rico Commonwealth   3548397     7    72

In [87]:
CensusStateDict=dfState[["state","STNAME"]].set_index("state").to_dict()["STNAME"]

In [95]:
CensusStateDict['05']

'Arkansas'

### 2010 Census data

In [91]:
# Get the variables JSON file for the Decennial census from 2010
vars=requests.get("https://api.census.gov/data/2010/dec/sf1/variables.json")
print(vars.json()['variables']["P012005"])
print(vars.json()['variables']["P012029"])
print(vars.json()['variables']["P012016"])
print(vars.json()['variables']["P012040"])
print(vars.json()['variables']["COUNTY"])

{'label': 'Total!!Male!!10 to 14 years', 'concept': 'SEX BY AGE', 'predicateType': 'int', 'group': 'P12', 'limit': 0}
{'label': 'Total!!Female!!10 to 14 years', 'concept': 'SEX BY AGE', 'predicateType': 'int', 'group': 'P12', 'limit': 0}
{'label': 'Total!!Male!!50 to 54 years', 'concept': 'SEX BY AGE', 'predicateType': 'int', 'group': 'P12', 'limit': 0}
{'label': 'Total!!Female!!50 to 54 years', 'concept': 'SEX BY AGE', 'predicateType': 'int', 'group': 'P12', 'limit': 0}
{'label': 'County (FIPS)', 'group': 'N/A', 'limit': 0}


In [93]:
server = "https://api.census.gov/data/2010/dec/sf1"
paramsIn = {"get": "P012005,P012029,P012016,P012040","for" : "COUNTY:*", "key": censusKey}
resp = requests.get(server,
                    params=paramsIn
                   )
if resp.status_code==200 : print("Good GET") 
print( type(resp.json() ) )
print( type(resp.json()[0])  ) 

Good GET
<class 'list'>
<class 'list'>


In [94]:
# the first entry here is the column heads but PCT012A017,PCT012A124 aren't informative
df=pd.DataFrame(resp.json()[1:])
df.columns=["male_10_to_14", "female_10_to_14","male_50_to_54", "female_50_to_54", "state","count"]
df

male_10_to_14 female_10_to_14 male_50_to_54 female_50_to_54 state count
0             4465            4171          4473            4517    05   131
1              677             671           502             487    05   133
2              581             522           583             623    05   135
3              385             353           486             463    05   137
4             1439            1370          1535            1592    05   139
...            ...             ...           ...             ...   ...   ...
3216          2857            2841          3164            3265    55   035
3217           115             103           248             206    55   037
3218          3398            3155          4006            3969    55   039
3219           298             279           339             364    55   041
3220          2851            2686          3139            3107    55   055

[3221 rows x 6 columns]

In [96]:
server = "https://api.census.gov/data/2010/dec/sf1"
paramsIn = {"get": "P012005,P012029,P012016,P012040","for" : "STATE:*", "key": censusKey}
resp = requests.get(server,
                    params=paramsIn
                   )
if resp.status_code==200 : print("Good GET") 
print( type(resp.json() ) )
print( type(resp.json()[0])  ) 

Good GET
<class 'list'>
<class 'list'>


In [97]:
df=pd.DataFrame(resp.json()[1:])
df.columns=["male_10_to_14","female_10_to_14","male_50_to_54", "female_50_to_54","state"]
df

male_10_to_14 female_10_to_14 male_50_to_54 female_50_to_54 state
0         163417          156238        168725          178760    01
1          26147           24669         29288           27012    02
2         228995          219669        202411          213113    04
3         100896           96663         99146          102576    05
4        1325915         1265015       1266543         1296009    06
5         156872          149964        161186          168143    22
6         145813          138341        156619          162836    21
7         170192          162462        184243          186820    08
8         122924          117341        138961          145364    09
9          28953           27895         31713           34285    10
10         12641           12400         18114           19050    11
11        578689          552158        650033          690258    12
12        352577          337107        324343          344248    13
13         41699           39840         48831           49147    15
14         60104           56851         52006           52971    16
15        449160          430288        455953          474832    17
16        230960          221211        233891          238931    18
17        103327           97577        111584          111660    19
18        102156           96728        101040          103394    20
19         40495           38518         54129           56827    23
20        193068          185961        211218          229401    24
21        207419          198194        241668          255333    25
22        345558          329658        376056          389396    26
23        180356          171986        200663          201032    27
24        106606          101642        100994          107613    28
25        203213          193712        218081          225725    29
26         31620           29504         38747           40064    30
27         62686           60020         65210           65025    31
28         93672           89501         92136           90601    32
29         43594           41026         55864           56533    33
30        300316          287019        328135          346545    34
31         72021           69670         71383           75787    35
32        619243          592213        687779          732149    36
33        322795          308309        323702          346191    37
34         20361           19429         25306           24971    38
35        396152          378547        434740          452317    39
36        130260          123404        129931          134438    40
37        124118          118435        134896          141300    41
38        405227          385924        484289          500352    42
39         32673           31151         39138           41912    44
40        152166          145120        156907          169755    45
41         27678           26282         29861           29538    46
42        215039          203902        222283          237066    47
43        962866          919017        827933          846936    48
44        117163          110788         75545           76588    49
45         19351           18286         25773           26720    50
46        261624          249622        288783          304062    51
47        224717          213516        245997          249299    53
48         55544           53501         70744           72488    54
49        192232          183695        218303          217823    55
50         18592           17363         22450           21655    56
51        137289          131282        110600          129221    72

In [98]:
# Add the state names using the dictionary we defined above and the map() method
df["state_name"]=df.state.map(CensusStateDict) 

In [99]:
df

male_10_to_14 female_10_to_14 male_50_to_54 female_50_to_54 state  \
0         163417          156238        168725          178760    01   
1          26147           24669         29288           27012    02   
2         228995          219669        202411          213113    04   
3         100896           96663         99146          102576    05   
4        1325915         1265015       1266543         1296009    06   
5         156872          149964        161186          168143    22   
6         145813          138341        156619          162836    21   
7         170192          162462        184243          186820    08   
8         122924          117341        138961          145364    09   
9          28953           27895         31713           34285    10   
10         12641           12400         18114           19050    11   
11        578689          552158        650033          690258    12   
12        352577          337107        324343          344248    13   
13         41699           39840         48831           49147    15   
14         60104           56851         52006           52971    16   
15        449160          430288        455953          474832    17   
16        230960          221211        233891          238931    18   
17        103327           97577        111584          111660    19   
18        102156           96728        101040          103394    20   
19         40495           38518         54129           56827    23   
20        193068          185961        211218          229401    24   
21        207419          198194        241668          255333    25   
22        345558          329658        376056          389396    26   
23        180356          171986        200663          201032    27   
24        106606          101642        100994          107613    28   
25        203213          193712        218081          225725    29   
26         31620           29504         38747           40064    30   
27         62686           60020         65210           65025    31   
28         93672           89501         92136           90601    32   
29         43594           41026         55864           56533    33   
30        300316          287019        328135          346545    34   
31         72021           69670         71383           75787    35   
32        619243          592213        687779          732149    36   
33        322795          308309        323702          346191    37   
34         20361           19429         25306           24971    38   
35        396152          378547        434740          452317    39   
36        130260          123404        129931          134438    40   
37        124118          118435        134896          141300    41   
38        405227          385924        484289          500352    42   
39         32673           31151         39138           41912    44   
40        152166          145120        156907          169755    45   
41         27678           26282         29861           29538    46   
42        215039          203902        222283          237066    47   
43        962866          919017        827933          846936    48   
44        117163          110788         75545           76588    49   
45         19351           18286         25773           26720    50   
46        261624          249622        288783          304062    51   
47        224717          213516        245997          249299    53   
48         55544           53501         70744           72488    54   
49        192232          183695        218303          217823    55   
50         18592           17363         22450           21655    56   
51        137289          131282        110600          129221    72   

                  state_name  
0                    Alabama  
1                     Alaska  
2                    Arizona  
3                   Arkansas  
4                 California  


In [100]:
# Here I change the data from strings to integers for the numeric fields
df=df.astype({'male_10_to_14': 'int32','female_10_to_14':'int32','male_50_to_54': 'int32','female_50_to_54':'int32'})

In [102]:
df["diff_young"]=df.male_10_to_14-df.female_10_to_14
df["diff_old"]=df.male_50_to_54-df.female_50_to_54

In [103]:
# Overall rate Men at 10-14
100*(df["male_10_to_14"].sum() )/(df["male_10_to_14"].sum()+df["female_10_to_14"].sum() )

51.1661951711957

In [ ]:
# Overall rate Men at 50-54
100*(df["male_50_to_54"].sum() )/(df["male_50_to_54"].sum()+df["female_50_to_54"].sum() )

In [ ]:
df["sex_ratio_10_to_14"]=100*df["male_10_to_14"]/(df["male_10_to_14"]+df["female_10_to_14"] )
df["sex_ratio_50_to_54"]=100*df["male_50_to_54"]/(df["male_50_to_54"]+df["female_50_to_54"] )
df["drop"]=df["sex_ratio_10_to_14"]-df["sex_ratio_50_to_54"]

In [ ]:
df[ ["state_name", "sex_ratio_10_to_14" ,"sex_ratio_50_to_54", "drop"] ].sort_values(by=['drop'])

### As a Package
---
Again, searching online, I quickly found a [python package](https://pypi.org/project/CensusData/) for engaging with this API, where they had several Jupyter notebooks in the `/docs`

In [104]:
import censusdata

In [105]:
# Download ACS 2011-2015 5-year estimates for Pittsburgh city, Pennsylvania on population size, median age, and median household income.
censusdata.download('acs5', 2015,censusdata.censusgeo([('state', '42'), ('place', '61000')]), ['B01001_001E', 'B01002_001E', 'B19013_001E'], key=censusKey)

B01001_001E  B01002_001E  \
Pittsburgh city, Pennsylvania: Summary level: 1...       305928         33.2   

                                                    B19013_001E  
Pittsburgh city, Pennsylvania: Summary level: 1...        40715

## Packages that interact with APIs
Many large companies will offer some type of API (though it may not be open to everyone). Searching around for these types of resources can be very useful!

In [106]:
from nba_api.stats.static import players
from nba_api.stats.endpoints import playergamelog, playerprofilev2, commonplayerinfo
from nba_api.stats.library.parameters import SeasonAll

In [107]:
player_list = players.get_active_players()

In [109]:
player_list[0]

{'id': 1630173,
 'full_name': 'Precious Achiuwa',
 'first_name': 'Precious',
 'last_name': 'Achiuwa',
 'is_active': True}

In [110]:
bamAdebayo=players.find_players_by_full_name('Bam Adebayo')[0]
bamAdebayo

{'id': 1628389,
 'full_name': 'Bam Adebayo',
 'first_name': 'Bam',
 'last_name': 'Adebayo',
 'is_active': True}

In [111]:
bamAdebayo['id']

1628389

In [112]:
bamPlayerInfo=commonplayerinfo.CommonPlayerInfo(player_id=bamAdebayo['id']).get_normalized_dict()
pd.DataFrame(bamPlayerInfo['CommonPlayerInfo'])

PERSON_ID FIRST_NAME LAST_NAME DISPLAY_FIRST_LAST DISPLAY_LAST_COMMA_FIRST  \
0    1628389        Bam   Adebayo        Bam Adebayo             Adebayo, Bam   

  DISPLAY_FI_LAST  PLAYER_SLUG            BIRTHDATE    SCHOOL COUNTRY  ...  \
0      B. Adebayo  bam-adebayo  1997-07-18T00:00:00  Kentucky     USA  ...   

    PLAYERCODE FROM_YEAR TO_YEAR  DLEAGUE_FLAG NBA_FLAG GAMES_PLAYED_FLAG  \
0  bam_adebayo      2017    2021             N        Y                 Y   

  DRAFT_YEAR DRAFT_ROUND  DRAFT_NUMBER GREATEST_75_FLAG  
0       2017           1            14                N  

[1 rows x 33 columns]

In [113]:
bamData=playergamelog.PlayerGameLog(player_id=bamAdebayo['id'],season=SeasonAll.all).get_data_frames()[0]
bamData.head()

SEASON_ID  Player_ID     Game_ID     GAME_DATE      MATCHUP WL  MIN  FGM  \
0     22021    1628389  0022101005  MAR 12, 2022  MIA vs. MIN  L   33    7   
1     22021    1628389  0022100998  MAR 11, 2022  MIA vs. CLE  W   33   11   
2     22021    1628389  0022100983  MAR 09, 2022  MIA vs. PHX  L   30    7   
3     22021    1628389  0022100970  MAR 07, 2022  MIA vs. HOU  W   30    8   
4     22021    1628389  0022100956  MAR 05, 2022  MIA vs. PHI  W   31    3   

   FGA  FG_PCT  ...  DREB  REB  AST  STL  BLK  TOV  PF  PTS  PLUS_MINUS  \
0   12   0.583  ...    10   12    4    3    0    4   2   19           0   
1   16   0.688  ...    15   17    1    3    1    3   4   30          13   
2   11   0.636  ...     6    6    5    1    1    4   1   17          -6   
3   15   0.533  ...     7   10    1    0    1    2   4   18          11   
4   10   0.300  ...     8   10    2    1    0    1   4    6           6   

   VIDEO_AVAILABLE  
0                1  
1                1  
2                1  
3                1  
4                1  

[5 rows x 27 columns]

And so now I can calculate his career average stats:

In [114]:
bamData[["PTS","REB","AST"]].mean()

PTS    13.253776
REB     8.283988
AST     3.486405
dtype: float64